In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from quantiphy import Quantity

In [2]:
def load_data(filename):
    return pd.read_csv(filename)

In [3]:
def parse_columns(data):
    data['subgroup'] = ((data['dut_setting'] != data['dut_setting'].shift(1)) | (data['dut'] != data['dut'].shift(1))).cumsum()
    data['nominal'] = data['dut_setting'].str.rsplit(' ', 1, expand=True).loc[:,0].apply(lambda r: float(Quantity(r)))
    data['ppm_dev'] = (data['ag3458a_2_ohm'] - data['nominal']) / data['nominal'] * 1e6

In [4]:
def plot_data(data):
    plot_df = data[(data['dut'] == 'SR1050-100k') & (data['dut_setting'].str.startswith('100 kOhm '))].loc[:,['dut_setting','ppm_dev']]
    plt.scatter(plot_df.index, plot_df['ppm_dev'], c=pd.factorize(plot_df['dut_setting'])[0])

In [5]:
def aggregate_by_subgroup(data):
    aggregated_data = data.groupby(['subgroup', 'dut', 'dut_setting'])[['dut', 'dut_setting', 'ppm_dev']].agg(['mean', 'std', 'count'])
    aggregated_data.columns = aggregated_data.columns.droplevel(0)
    return aggregated_data

In [6]:
def get_best_data(aggregated_data):
    last_dut_setting_subgroups = aggregated_data.reset_index().groupby(['dut', 'dut_setting'])['subgroup'].max().reset_index()
    joined = pd.merge(aggregated_data.reset_index(), last_dut_setting_subgroups, on=['dut', 'dut_setting'], how='left', suffixes=['', '_last'])
    best_data = joined[joined['subgroup'] == joined['subgroup_last']].drop(columns=['subgroup_last'])
    best_data['resistors'] = best_data['dut_setting'].str.replace(r'R(\d+)[AB]-R\1[AB]', r'R\1').str.replace(r'R(\d+)[AB]-R(\d+)[AB]', r'R\1-R\2')
    return best_data

In [35]:
def get_single_resistor_data(best_data):
    combined_data = best_data.groupby(['dut', 'resistors']).agg({'mean': 'mean', 'count': 'sum', 'std': lambda s: np.sqrt(s.apply(lambda v: v**2).sum())})
    indiv_res_data = combined_data[~(combined_data.index.get_level_values(1).str.startswith('11') | combined_data.index.get_level_values(1).str.startswith('1.'))]
    avg_indiv_res = indiv_res_data.groupby('dut').agg({'mean': 'mean', 'count': 'sum', 'std': lambda s: np.sqrt(s.apply(lambda v: v**2).sum())})
    single_with_avg = pd.merge(indiv_res_data.reset_index(), avg_indiv_res, on='dut', suffixes=['_single', '_avg']).set_index(['dut', 'resistors'])
    single_with_avg['rel_difference'] = single_with_avg['mean_single'] - single_with_avg['mean_avg']
    return single_with_avg

In [36]:
def get_min_max_deviation(single_resistor_data):
    return single_resistor_data.groupby('dut').agg({'rel_difference': ['min', 'max']})

def filter_temp2(data):
    return data[data.temp_2.isna()]

In [37]:
sr1050_data = load_data('ks3458a-resistance-decade-log-sr1050.csv')
parse_columns(sr1050_data)
sr1050_data = filter_temp2(sr1050_data)
sr1050_aggregated_data = aggregate_by_subgroup(sr1050_data)
sr1050_best_data = get_best_data(sr1050_aggregated_data)
sr1050_single_resistor_data = get_single_resistor_data(sr1050_best_data)
sr1050_min_max_data = get_min_max_deviation(sr1050_single_resistor_data)

/tmp/ipykernel_29029/636476981.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  best_data['resistors'] = best_data['dut_setting'].str.replace(r'R(\d+)[AB]-R\1[AB]', r'R\1').str.replace(r'R(\d+)[AB]-R(\d+)[AB]', r'R\1-R\2')


In [38]:
sr1050_single_resistor_data

mean_single  count_single  std_single   mean_avg  \
dut         resistors                                                        
SR1050-100k 100 kOhm R1     26.273806            45    0.210701  21.810608   
            100 kOhm R10    20.374654            28    0.082101  21.810608   
            100 kOhm R11    12.010425            31    0.149323  21.810608   
            100 kOhm R2     24.194045            41    0.101913  21.810608   
            100 kOhm R3     19.076588            50    0.193271  21.810608   
            100 kOhm R4     25.305906            30    0.151622  21.810608   
            100 kOhm R5     25.908494            52    0.142297  21.810608   
            100 kOhm R6     19.216829           170    0.282926  21.810608   
            100 kOhm R7     21.924876            52    0.142849  21.810608   
            100 kOhm R8     25.112287           122    0.256393  21.810608   
            100 kOhm R9     20.518776            37    0.161835  21.810608   
SR1050-10M  10 MOhm R1      25.290734            39    0.716560  29.469114   
            10 MOhm R10     24.381849            39    1.687082  29.469114   
            10 MOhm R11     28.202486            56    0.690488  29.469114   
            10 MOhm R2      28.598397            43   17.113512  29.469114   
            10 MOhm R3      25.900260            40    0.981521  29.469114   
            10 MOhm R4      29.055521            39    1.449563  29.469114   
            10 MOhm R5      28.760912            34    1.031286  29.469114   
            10 MOhm R6      31.412721            46    1.393280  29.469114   
            10 MOhm R7      35.456614            38    0.935673  29.469114   
            10 MOhm R8      34.074595            49    5.837464  29.469114   
            10 MOhm R9      33.026167            99    1.823386  29.469114   
SR1050-1M   1 MOhm R1       29.277551            56    1.735137  21.344514   
            1 MOhm R10      20.369033           110    1.027152  21.344514   
            1 MOhm R11      17.955812           112    1.849230  21.344514   
            1 MOhm R2       18.845626           118    0.652159  21.344514   
            1 MOhm R3       12.262841            58    0.666234  21.344514   
            1 MOhm R4       21.634087            52    1.587555  21.344514   
            1 MOhm R5       22.422857            43    0.636617  21.344514   
            1 MOhm R6       25.961819            54    0.614937  21.344514   
            1 MOhm R7       24.197639            54    0.664300  21.344514   
            1 MOhm R8       17.616142            62    3.299309  21.344514   
            1 MOhm R9       24.246246            34    2.174635  21.344514   

                          count_avg    std_avg  rel_difference  
dut         resistors                                           
SR1050-100k 100 kOhm R1         658   0.597320        4.463198  
            100 kOhm R10        658   0.597320       -1.435954  
            100 kOhm R11        658   0.597320       -9.800183  
            100 kOhm R2         658   0.597320        2.383438  
            100 kOhm R3         658   0.597320       -2.734020  
            100 kOhm R4         658   0.597320        3.495298  
            100 kOhm R5         658   0.597320        4.097886  
            100 kOhm R6         658   0.597320       -2.593779  
            100 kOhm R7         658   0.597320        0.114268  
            100 kOhm R8         658   0.597320        3.301679  
            100 kOhm R9         658   0.597320       -1.291832  
SR1050-10M  10 MOhm R1          522  18.467657       -4.178380  
            10 MOhm R10         522  18.467657       -5.087265  
            10 MOhm R11         522  18.467657       -1.266628  
            10 MOhm R2          522  18.467657       -0.870717  
            10 MOhm R3          522  18.467657       -3.568854  
            10 MOhm R4          522  18.467657       -0.413593  
            10 MOhm R5          522  18.467657       -0.7082

In [23]:
sr1050_min_max_data

rel_difference          
                       min       max
dut                                 
SR1050-100k      -3.714038  3.483179
SR1050-10M       -5.213928  5.860837
SR1050-1M        -9.420543  7.594167

In [32]:
sr1010_data = load_data('ks3458a-resistance-decade-log-sr1010.csv')
parse_columns(sr1010_data)
sr1010_aggregated_data = aggregate_by_subgroup(sr1010_data)
sr1010_best_data = get_best_data(sr1010_aggregated_data)
sr1010_single_resistor_data = get_single_resistor_data(sr1010_best_data)
sr1010_min_max_data = get_min_max_deviation(sr1010_single_resistor_data)

/tmp/ipykernel_29029/636476981.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  best_data['resistors'] = best_data['dut_setting'].str.replace(r'R(\d+)[AB]-R\1[AB]', r'R\1').str.replace(r'R(\d+)[AB]-R(\d+)[AB]', r'R\1-R\2')


In [33]:
sr1010_single_resistor_data

mean_single  count_single  std_single   mean_avg  \
dut        resistors                                                       
SR1010-10k 10 kOhm R1     24.475625            16    0.130424  26.634920   
           10 kOhm R10    31.851435            92    0.585753  26.634920   
           10 kOhm R11    33.749861            72    0.422349  26.634920   
           10 kOhm R2     26.228206            34    0.088493  26.634920   
           10 kOhm R3     28.029254            59    0.105741  26.634920   
           10 kOhm R4     22.577500            52    0.232601  26.634920   
           10 kOhm R5     20.697882            34    0.143908  26.634920   
           10 kOhm R6     21.997460           100    0.315181  26.634920   
           10 kOhm R7     29.502344            96    0.133074  26.634920   
           10 kOhm R8     25.150899            69    0.169435  26.634920   
           10 kOhm R9     26.942658            38    0.117177  26.634920   
           10 kOmh R12    28.415917            72    0.122583  26.634920   
SR1010-1k  1 kOhm R1      16.720923            65    0.103519  19.589646   
           1 kOhm R10     21.554306            49    0.105076  19.589646   
           1 kOhm R11     21.272957            23    0.101938  19.589646   
           1 kOhm R12     23.287650            20    0.089166  19.589646   
           1 kOhm R2      11.441554            65    0.127582  19.589646   
           1 kOhm R3      15.155760            25    0.136575  19.589646   
           1 kOhm R4      10.981588            34    0.154038  19.589646   
           1 kOhm R5      25.789375            24    0.198030  19.589646   
           1 kOhm R6      15.092806            36    0.116483  19.589646   
           1 kOhm R7      16.856469            32    0.122877  19.589646   
           1 kOhm R8      35.730273            33    0.100891  19.589646   
           1 kOhm R9      21.192089            45    0.099421  19.589646   

                        count_avg   std_avg  rel_difference  
dut        resistors                                         
SR1010-10k 10 kOhm R1         734  0.898239       -2.159295  
           10 kOhm R10        734  0.898239        5.216515  
           10 kOhm R11        734  0.898239        7.114941  
           10 kOhm R2         734  0.898239       -0.406714  
           10 kOhm R3         734  0.898239        1.394334  
           10 kOhm R4         734  0.898239       -4.057420  
           10 kOhm R5         734  0.898239       -5.937038  
           10 kOhm R6         734  0.898239       -4.637460  
           10 kOhm R7         734  0.898239        2.867424  
           10 kOhm R8         734  0.898239       -1.484021  
           10 kOhm R9         734  0.898239        0.307738  
           10 kOmh R12        734  0.898239        1.780997  
SR1010-1k  1 kOhm R1          451  0.432091       -2.868723  
           1 kOhm R10         451  0.432091        1.964660  
           1 kOhm R11         451  0.432091        1.683311  
           1 kOhm R12         451  0.432091        3.698004  
           1 kOhm R2          451  0.432091       -8.148092  
           1 kOhm R3          451  0.432091       -4.433886  
           1 kOhm R4          451  0.432091       -8.608057  
           1 kOhm R5          451  0.432091        6.199729  
           1 kOhm R6          451  0.432091       -4.496840  
           1 kOhm R7          451  0.432091       -2.733177  
           1 kOhm R8          451  0.432091       16.140627  
           1 kOhm R9          451  0.432091        1.602443

In [34]:
sr1010_min_max_data

rel_difference           
                      min        max
dut                                 
SR1010-10k      -5.937038   7.114941
SR1010-1k       -8.608057  16.140627